# Performance Tests of Apache Spark-Based DC2 Run 1.1 Object Catalog Access
Author: **Julien Peloton [@JulienPeloton](https://github.com/JulienPeloton)**  
Last Run: **2018-11-22**  
See also: [issue/249](https://github.com/LSSTDESC/DC2-production/issues/249)

The purpose of this notebook is twofold: introduce Apache Spark and test performance of data manipulations of the static coadd catalogs. More benchmarks can be found on the companion notebook (*_appendix.ipynb)


## Before starting...

**What is Apache Spark?**

I'm glad you asked! [Apache Spark](http://spark.apache.org/) is a cluster computing framework, that is a set of tools to perform computation on a network of many machines. Spark started in 2009 as a research project, and it had a huge success so far in the industry. It is based on the so-called MapReduce cluster computing paradigm, popularized by the Hadoop framework using implicit data parallelism and fault tolerance. 

**Where to find information on running Spark at NERSC?**

Most of what you need for interactive and batch jobs is at [spark-distributed-analytic-framework](https://www.nersc.gov/users/data-analytics/data-analytics-2/spark-distributed-analytic-framework/).

For JupyterLab use, see below.

**Where this Notebook is intended to be run?**

These tests were conducted on NERSC through the https://jupyter-dev.nersc.gov interface.

**What is needed to run this Notebook at NERSC?**

1. You need an account at NERSC, and access to the DESC allocation.

2. This Notebook requires a pyspark kernel. The easiest way is to use the `desc-pyspark` kernel (see [LSSTDESC/desc-spark](https://github.com/LSSTDESC/desc-spark#working-at-nersc-jupyterlab) or [LSSTDESC/jupyter-kernels](https://github.com/LSSTDESC/jupyter-kernels) for more information)
If you encounter problem with this kernel, let [me](https://github.com/LSSTDESC/DC2_Repo/issues/new?body=@JulienPeloton) know.

**Where is the data used in this Notebook?**

Data used can be found at 

```
/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/summary/
```

Apache Spark can read a large number of data formats (Parquet, Avro, text) but officially neither FITS nor HDF5 are supported. We developed a solution for FITS files ([spark-fits](https://github.com/astrolabsoftware/spark-fits), Scala/Java/Python/R API), but as far as I know there is no Python-friendly connector for HDF5. Therefore we will focus only on Parquet and FITS in this Notebook.

**Note concerning resources**

```
The large-memory login node used by https://jupyter-dev.nersc.gov/
is a shared resource, so please be careful not to use too many CPUs
or too much memory.

That means avoid using `--master local[*]` in your kernel, but limit
the resources to a few core. Typically `--master local[4]` is enough for
prototyping a program.

Then to scale the analysis, the best is to switch to batch mode! 
There, no limit!
```

This is already taken care for you in the DESC kernel setup script (from desc-pyspark), but keep this in mind if you use a custom kernel.

## Loading the data

We will follow what is done on the Dask Notebook ([hdf5, pandas](https://github.com/LSSTDESC/DC2-production/blob/master/Notebooks/object_catalog_performance_dask.ipynb), [parquet](https://github.com/LSSTDESC/DC2-production/blob/master/Notebooks/object_catalog_performance_dask_parquet.ipynb)).
We will first focus on one `patch` (4850) with all `tracts`.

### Disclaimer

Apache Spark, is meant to be primarily used in a context of _big data_.
One of its strength is its scalability, namely its capability of using the same
piece of code regardless the underlying data volume. The performance of the code will
then only depend on the resource used. 
E.g. for tasks without communications, execution time will be linear with data or resource.

Keep in mind:
- For small volume of data (< 10 GB), you will hit Spark noise and burning time.
- Spark is written in Scala, which is certainly not as specialised as C++ could be. Therefore for small volume of data, there is a chance an algorithm in Scala (Spark) would be slower than its C++ counterpart. But the Spark one is meant to run on TB of data _as it was written_ for MB of data - which is probably not the case for the C++.
- Once the data loaded, you can decide to keep it in memory (distributed among the executors). The next iterations will then go super fast (typically disk I/O throughput is o(100) MB/s while RAM is o(10) GB/s).

So in this example, the one patch test is likely to lessen Spark performance (volume is just few GB here), and these tests must also be ran on hundreds of GB of data.

In [1]:
import os

base_dir = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/summary'

# Load one patch, all tracts
datafile = os.path.join(base_dir, 'dpdd_object.parquet')
print("Data will be read from: \n", datafile)

Data will be read from: 
 /global/projecta/projectdirs/lsst/global/in2p3/Run1.1/summary/dpdd_object.parquet


## Loading data into a DataFrame

Let's initialise Spark and load the data into a DataFrame. We will first focus on the `parquet` data format.

In [2]:
from pyspark.sql import SparkSession

# Initialise our Spark session
spark = SparkSession.builder.getOrCreate()

# Read the data as DataFrame
df = spark.read.format("parquet").load(datafile)

# Check what we have in the file
df.printSchema()

root
 |-- magerr_i: double (nullable = true)
 |-- psFlux_i: double (nullable = true)
 |-- Ixx_r: double (nullable = true)
 |-- mag_i_cModel: double (nullable = true)
 |-- IxxPSF_u: double (nullable = true)
 |-- magerr_r: double (nullable = true)
 |-- psf_fwhm_i: double (nullable = true)
 |-- psf_fwhm_r: double (nullable = true)
 |-- Ixx: double (nullable = true)
 |-- magerr_g_cModel: double (nullable = true)
 |-- I_flag_y: boolean (nullable = true)
 |-- Iyy_z: double (nullable = true)
 |-- IxyPSF_i: double (nullable = true)
 |-- Ixx_z: double (nullable = true)
 |-- magerr_u_cModel: double (nullable = true)
 |-- IxyPSF: double (nullable = true)
 |-- snr_u_cModel: double (nullable = true)
 |-- IxxPSF_y: double (nullable = true)
 |-- psFlux_flag_i: boolean (nullable = true)
 |-- IyyPSF_g: double (nullable = true)
 |-- Ixy: double (nullable = true)
 |-- magerr_y: double (nullable = true)
 |-- psFlux_g: double (nullable = true)
 |-- snr_y_cModel: double (nullable = true)
 |-- Ixy_z: double 

In [3]:
# Get number of elements
print("All tracts DataFrame has length:", df.count())
%timeit c = df.count()

All tracts DataFrame has length: 6892380
The slowest run took 4.37 times longer than the fastest. This could mean that an intermediate result is being cached.
2.54 s ± 1.44 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Some statistics about the data

Let's play with the data. We will see how to compute statistics (mean, std, etc...) and ...

In [4]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col

def stat_one_col(df: DataFrame, colname: str) -> DataFrame:
    """ Return some statistics about one DataFrame Column.
    Statistics include: count, mean, stddev, min, max.
    
    Parameters
    ----------
    df : DataFrame
        Spark DataFrame
    colname : str
        Name of the Column for which we want the statistics
        
    Returns
    ----------
    out : DataFrame
        DataFrame containing statistics about the Column.
    """
    return df.select(colname).describe()

def stat_diff_col(df: DataFrame, colname_1: str, colname_2: str) -> DataFrame:
    """ Return some statistics about the difference of 
    two DataFrame Columns.
    Statistics include: count, mean, stddev, min, max.
    
    Parameters
    ----------
    df : DataFrame
        Spark DataFrame
    colname_1 : str
        Name of the first Column
    colname_2 : str
        Name of the second Column
        
    Returns
    ----------
    out : DataFrame
        DataFrame containing statistics about the Columns difference.
    """
    return df.select(col(colname_1) - col(colname_2)).describe()

In [5]:
# Get statistics about one column
stat_one_col(df, 'mag_g').show()
%timeit c = stat_one_col(df, 'mag_g').collect()

# Get statistics for the difference of two columns
stat_diff_col(df, 'mag_g', 'mag_r').show()
%timeit d = stat_diff_col(df, 'mag_g', 'mag_r').collect()

+-------+------------------+
|summary|             mag_g|
+-------+------------------+
|  count|           5698019|
|   mean|25.721391706804777|
| stddev|  2.16444185648169|
|    min|14.431595512576623|
|    max| 52.39393263760668|
+-------+------------------+

1.53 s ± 124 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
+-------+------------------+
|summary|   (mag_g - mag_r)|
+-------+------------------+
|  count|           4172439|
|   mean|0.5274674368917402|
| stddev|1.6067276263035577|
|    min|-28.13517076318348|
|    max|33.316740496333026|
+-------+------------------+

The slowest run took 5.29 times longer than the fastest. This could mean that an intermediate result is being cached.
3.91 s ± 2.47 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


It takes roughly 1 second to produce statistics on the full dataset (6 million rows, 4.5 GB). This has to be compared to the 52 seconds for Dask to compute the `mean` using the same resource (4 CPU). Note that we didn't explicitly asked to put the data in cache (would be much faster). Note also that the number of elements for each column used to produce statistics varies (`count`). This is due to the fact that `NaN` are discarded.

I also ran this test with more resource (i.e. more CPUs), and the execution time gets smaller. Not linearly, as we hit Spark burning time (not enough data), but e.g. a few hundred of millisecond with 32 CPU.

## Reducing, playing, plotting

You can always go back to pandas world by using the `toPandas()` method. But be careful, if you do that on the full dataframe you will destroy your driver for sure! Spark DataFrames are abstractions of arbitrary amount of data. Invoking `toPandas()` triggers an action, and objects to transfer will be materialised. Imagine TB of data suddenly flowing to your driver...

So transfer only subset of the full Spark DataFrame to pandas. In pyspark there are several ways of selecting only subset of data: via SQL expression or DataFrame methods (which are somewhat related to each other...): 

In [6]:
# Subset of columns of interest
cols = "mag_g, mag_r, mag_i, magerr_g, magerr_r, magerr_i, extendedness"

# SQL - register first the DataFrame
df.createOrReplaceTempView("full_tract")

# Keeps only columns with 0.0 < magerr_g < 0.3
sql_command = """
    SELECT {}
    FROM full_tract 
    WHERE 
        magerr_g > 0 AND magerr_g < 0.3
""".format(cols)

# Execute the expression - return a DataFrame
df_sub = spark.sql(sql_command)
print("Number of elements selected: ", df_sub.count())
df_sub.show()

# Note that we could have done otherwise:
# Example with select.where
df.select(cols.split(", ")).where("magerr_g > 0 AND magerr_g < 0.3")
# Example with select.filter
df.select(cols.split(", ")).filter("magerr_g > 0 AND magerr_g < 0.3")

Number of elements selected:  3321654
+------------------+-----+------------------+--------------------+--------+-------------------+------------+
|             mag_g|mag_r|             mag_i|            magerr_g|magerr_r|           magerr_i|extendedness|
+------------------+-----+------------------+--------------------+--------+-------------------+------------+
| 24.87676944063096| null| 23.64673432826903| 0.17546297070343575|    null|0.13097482849456088|         1.0|
|24.939799594405795| null|24.450642477068754|  0.1859198552723055|    null| 0.2702956478950862|         1.0|
|22.606359735849033| null|21.783006265387925|0.024030475892070212|    null|0.02584801396172265|         1.0|
| 24.60100335942729| null| 23.78327461507407| 0.13540762291556163|    null|0.14721246644511554|         1.0|
| 24.90555337049352| null|23.914387997196656| 0.17788127357700817|    null| 0.1641571882704487|         1.0|
|25.192049644404587| null|23.960523492790774|  0.2339227977627664|    null|0.1722461689861

DataFrame[mag_g: double, mag_r: double, mag_i: double, magerr_g: double, magerr_r: double, magerr_i: double, extendedness: double]

3000 objects, 7 columns - that's more reasonnable than the million ones! We can go to Pandas and play for example with your favourite plotting library such as seaborn, matplotlib, and so on:

In [7]:
df_pandas = df_sub.toPandas()
df_pandas

,mag_g,mag_r,mag_i,magerr_g,magerr_r,magerr_i,extendedness
0,24.876769,NaN,23.646734,0.175463,NaN,0.130975,1.0
1,24.939800,NaN,24.450642,0.185920,NaN,0.270296,1.0
2,22.606360,NaN,21.783006,0.024030,NaN,0.025848,1.0
3,24.601003,NaN,23.783275,0.135408,NaN,0.147212,1.0
4,24.905553,NaN,23.914388,0.177881,NaN,0.164157,1.0
5,25.192050,NaN,23.960523,0.233923,NaN,0.172246,0.0
6,23.468926,NaN,22.029117,0.049246,NaN,0.059986,1.0
7,24.443414,NaN,23.754252,0.164376,NaN,0.142844,1.0
8,24.930234,NaN,23.606672,0.182960,NaN,0.124861,1.0
9,23.582033,NaN,22.970755,0.053768,NaN,0.130812,1.0


## Data quality

**87% valid entries, 13% NaN**

In [8]:
from pyspark.sql.functions import col,sum

# Show the number of valid entries
df_entries = df.select(*(sum(col(c).isNotNull().cast("int")).alias(c) for c in df.columns))
entries = df_entries.collect()

In [9]:
import numpy as np

ini_len = df.count()
print("Input number of entries: {}".format(ini_len))
print("Total yield: {}%".format(np.sum(entries) / (ini_len * len(df.columns)) * 100))
for col, entry in zip(df.columns, entries[0]):
    print("{}: {} ({:.1f}%)".format(col, entry, entry/ini_len*100))

Input number of entries: 6892380
Total yield: 87.42589520552244%
magerr_i: 5272546 (76.5%)
psFlux_i: 6142802 (89.1%)
Ixx_r: 4997285 (72.5%)
mag_i_cModel: 5196764 (75.4%)
IxxPSF_u: 6791749 (98.5%)
magerr_r: 4819340 (69.9%)
psf_fwhm_i: 6295438 (91.3%)
psf_fwhm_r: 5781874 (83.9%)
Ixx: 4252164 (61.7%)
magerr_g_cModel: 6568933 (95.3%)
I_flag_y: 6892380 (100.0%)
Iyy_z: 5209439 (75.6%)
IxyPSF_i: 6295438 (91.3%)
Ixx_z: 5209439 (75.6%)
magerr_u_cModel: 6510048 (94.5%)
IxyPSF: 6892038 (100.0%)
snr_u_cModel: 6510048 (94.5%)
IxxPSF_y: 6342770 (92.0%)
psFlux_flag_i: 6892380 (100.0%)
IyyPSF_g: 6776910 (98.3%)
Ixy: 4252164 (61.7%)
magerr_y: 5040748 (73.1%)
psFlux_g: 6740025 (97.8%)
snr_y_cModel: 6045334 (87.7%)
Ixy_z: 5209439 (75.6%)
psFlux_flag_r: 6892380 (100.0%)
Iyy_g: 5556470 (80.6%)
psFluxErr_r: 5600369 (81.3%)
Ixx_i: 5306107 (77.0%)
snr_z_cModel: 5804192 (84.2%)
psf_fwhm_g: 6776910 (98.3%)
Ixx_y: 5036312 (73.1%)
I_flag: 6892380 (100.0%)
magerr_z: 5113937 (74.2%)
I_flag_i: 6892380 (100.0%)
IyyPS